In [1]:
import sys
sys.path.append('C:/Users/Kai/Desktop/vis_scBERT/DNABERT_2/')
from get_gene_name_and_seq import *

# Get NT sequence from serval patheways of cmiu

In [2]:
raw='''
00190  Oxidative phosphorylation
00680  Methane metabolism
00910  Nitrogen metabolism
00920  Sulfur metabolism
'''
pathway_list = clean_raw_pathway_str(raw, 'cmiu')
cmiu_dic = pathway2NTseq_cmiu(pathway_list)

Getting NT sequences: 100%|██████████| 4/4 [01:12<00:00, 18.03s/it]


In [8]:
for k,v in cmiu_dic.items():
    print(k, len(v))

cmiu00190 12
cmiu00680 28
cmiu00910 7
cmiu00920 6
